<a href="https://colab.research.google.com/github/piggyatbaqaqi/skol/blob/master/IST691/workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SKOL III: Feature extraction

created by:
* La Monte Yarroll
* Padmaja Kurmaddali
* Patrick Le

# **Mapping Data Files: Google Drive**

In [1]:
import os, sys
from pathlib import Path
from google.colab import drive
%cd /content
content = Path('/content')
skol = content / 'drive/My Drive/SKOL'
piggyatbaqaqi = skol / 'github.com/piggyatbaqaqi'
drive.mount(str(content / "drive"), force_remount=True)
cache_path = content / 'cache'
ollama_cache_path = content / 'ollama_cache'
nb_path = content / 'packages'
if not os.path.exists(nb_path):
  nb_path.symlink_to(skol / 'packages')
skol_client = content / 'skol'
if not os.path.exists(skol_client):
  skol_client.symlink_to(piggyatbaqaqi / 'skol')
if not os.path.exists(cache_path):
  cache_path.symlink_to(skol / 'pip_cache')
if not os.path.exists(ollama_cache_path):
  ollama_cache_path.symlink_to(skol / 'ollama_cache')
os.environ['OLLAMA_MODELS'] = str(ollama_cache_path)
sys.path.insert(0, str(nb_path))
sys.path.insert(0, str(piggyatbaqaqi / 'skol'))

/content
Mounted at /content/drive


In [2]:
!ls -l /content/
!file /content
print(sys.path)

total 8
lrwxrwxrwx 1 root root   38 Jun  6 14:12 cache -> '/content/drive/My Drive/SKOL/pip_cache'
drwx------ 7 root root 4096 Jun  6 15:37 drive
lrwxrwxrwx 1 root root   41 Jun  6 14:12 ollama_cache -> '/content/drive/My Drive/SKOL/ollama_cache'
lrwxrwxrwx 1 root root   37 Jun  6 14:12 packages -> '/content/drive/My Drive/SKOL/packages'
drwxr-xr-x 1 root root 4096 Jun  4 21:29 sample_data
lrwxrwxrwx 1 root root   58 Jun  6 14:12 skol -> '/content/drive/My Drive/SKOL/github.com/piggyatbaqaqi/skol'
/content: directory
['/content/drive/My Drive/SKOL/github.com/piggyatbaqaqi/skol', '/content/packages', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython']


## Set up git clients

In [3]:
if not os.path.exists(piggyatbaqaqi):
  %mkdir -p $piggyatbaqaqi
if not os.path.exists(piggyatbaqaqi / 'skol'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/skol.git
sys.path.insert(0, piggyatbaqaqi / 'skol')
if not os.path.exists(piggyatbaqaqi / 'dr-drafts-mycosearch'):
  %cd $piggyatbaqaqi
  !git clone https://github.com/piggyatbaqaqi/dr-drafts-mycosearch.git
workdir = skol / 'IST691'
%cd $workdir

/content/drive/My Drive/SKOL/IST691


In [4]:
# install PySpark
! pip install --cache-dir=$cache_path --target=$nb_path pyspark
! pip install --cache-dir=$cache_path --target=$nb_path sparknlp ollama

  Using cached pyspark-4.0.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.7.5 requires pyspark[connect]~=3.5.1, but you have pyspark 4.0.0 which is incompatible.
  Using cached sparknlp-1.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached ollama-0.5.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached spark_nlp-6.0.2-py2.py3-none-any.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.5-py3-none-any.whl.metadata (67 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.4.26-py3-none-any.w

In [5]:
#import needed modules
import os
import glob
from typing import List
from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import (
    input_file_name, collect_list, concat_ws, col, udf,
    explode, collect_list, regexp_extract, regexp_replace,
    split, flatten, transform, concat)
from pyspark.sql.types import ArrayType, StringType

from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import (
    Tokenizer, HashingTF, IDF, StringIndexer, CountVectorizer,
    PCA, VectorAssembler)
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import split, row_number, min, expr, struct
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
from pyspark.sql.types import DoubleType, StructField, StructType

SKOL Data manipulation library

In [6]:
from finder import read_files, parse_annotated, target_classes
from label import Label
from taxon import Taxon, group_paragraphs

SEED=12345
default_label = Label('Misc-exposition')
keep_labels = [Label('Description'), Label('Nomenclature')]

# **Checking the file counts in the directories**

In [7]:
raw_directory_path = skol / 'raw_2025_02_05/'
ann_directory_path = skol / 'annotated_2025_02_27/journals'

## Checking the file counts in the directories

In [8]:
# Function that reports all the txt files under a Google Drive folder path
def listFiles(folder: str) -> List[str]:
  # List all files in the folder
  try:
      files = [file for file in glob.glob(f'{folder}/**/*.txt*', recursive=True) if 'Sydowia' not in file]
      return files
  except FileNotFoundError:
      print(f"Folder '{folder}' not found.")
  except PermissionError:
      print(f"Permission denied to access folder '{folder}'.")

In [9]:
# check files in raw directory
listFiles(raw_directory_path)[:10]

['/content/drive/My Drive/SKOL/raw_2025_02_05/authors/FungalNameAuthors.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753420.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753372.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753466.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753104.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753322.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/mycologiaeuropa00persgoog.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753038.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753160.txt',
 '/content/drive/My Drive/SKOL/raw_2025_02_05/Mycologia/3753583.txt']

In [10]:
# check files in annotated directory
training_files = listFiles(ann_directory_path)
training_files[:10]

['/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol057/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol054/n1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s17.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s29.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s30.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s7.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s21.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s13.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s1.txt.ann',
 '/content/drive/My Drive/SKOL/annotated_2025_02_27/journals/Mycotaxon/Vol118/s46.txt.ann']

In [11]:
len(training_files)

190

In [12]:
paragraphs = list(parse_annotated(read_files(random.sample(training_files, 20))))

In [13]:
relabeled = list(target_classes(default=default_label, keep=keep_labels, paragraphs=paragraphs))

In [14]:
print(f'len(relabeled): {len(relabeled)}')

len(relabeled): 10802


In [15]:
df = pd.DataFrame({
    'filename': [pp.filename for pp in relabeled],
    'label': [pp.top_label().label if pp.top_label() else None for pp in relabeled],
    'paragraph_number': [pp.paragraph_number for pp in relabeled],
    'page_number': [pp.page_number for pp in relabeled],
    'empirical_page_number': [pp.empirical_page_number for pp in relabeled],
    'line_number': [pp.first_line.line_number if pp.first_line else None for pp in relabeled],
    'body': [str(pp) for pp in relabeled]
})
df.label = pd.Categorical(df.label)
df['label_code'] = df.label.cat.codes

In [16]:
df.groupby('label', observed=True).nunique()

,filename,paragraph_number,page_number,empirical_page_number,line_number,body,label_code
label,,,,,,,
Description,15,670,131,185,63,595,1
Misc-exposition,20,9637,166,406,194,4831,1
Nomenclature,16,495,108,149,61,495,1


In [17]:
#load in library to open terminal inside google colab
# !pip install  --cache-dir=$cache_path --target=$nb_path colab-xterm
# %load_ext colabxterm


In [18]:
!ls /usr/local/lib/ollama

cuda_v11		  libggml-cpu-haswell.so      libggml-cpu-sse42.so
cuda_v12		  libggml-cpu-icelake.so      libggml-cpu-x64.so
libggml-base.so		  libggml-cpu-sandybridge.so
libggml-cpu-alderlake.so  libggml-cpu-skylakex.so


In [19]:
import subprocess
import time

import ollama
#open up terminal
# %xterm
#once open, first time do:
if not os.path.exists("/usr/local/lib/ollama"):
  !curl https://ollama.ai/install.sh | sh
ollama_server = subprocess.Popen(["ollama", "serve"])
time.sleep(5)  # Let the server come all the way up.
#then start the server with ollama serve &
#first time also will need to pull in a ollama version using ollama pull mistral
# preferred_model = 'gemma3:12b'
preferred_model = 'mistral'
found = False
for _, l in ollama.list():
  for m in l:
    if m.model.startswith(preferred_model):
      found = True
      break
if not found:
  print(f"Pulling model {preferred_model}")
  !ollama pull $preferred_model


In [20]:
#check here if ollama has a version and can be used, will say model you pulled
!ollama list
# !ollama pull mistral
# !ollama pull gemma3:12b

NAME              ID              SIZE      MODIFIED    
mistral:latest    f974a74358d6    4.1 GB    2 weeks ago    
gemma3:12b        f4031aab637d    8.1 GB    2 weeks ago    


In [21]:
    import ollama

    response = ollama.generate(
        model=preferred_model,
        prompt="What is the capital of France?"
    )
    print(response["response"])

 The capital of France is Paris.


In [22]:
#test if ollama call works
import ollama
response = ollama.generate(model=preferred_model, prompt='Why is the sky blue?')
print(response['response'])

 The blue color of the sky is due to a process called Rayleigh scattering. In this process, short-wavelength light (blue and violet) is scattered more than other colors by molecules in the Earth's atmosphere because they are smaller than the wavelengths of this light. This scattered blue and violet light reaches our eyes from all directions, making the sky appear blue to us. However, we see the sky as blue rather than violet due to a phenomenon called the Moon illusion – our brain perceives the sky as more expansive when it's near the horizon, causing us to interpret it as less saturated with color and thus blue, rather than violet.


In [ ]:
grouped = group_paragraphs(relabeled)
prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Translate Latin paragraphs to English before any other processing.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
'''
for i, tax in enumerate(grouped):
  # if i > 10:
  #   break
  print(f'Send to LLM:\n\n{tax.as_row()["description"]}')
  #sample message to turn data into json format
  response = ollama.chat(model=preferred_model, messages=[{
     'role': 'user',
     'content': f'{prompt}\n\n{tax.as_row()["description"]}',},
  ])
  print('Result:')
  print(response['message']['content'])

Streaming output truncated to the last 5000 lines.
            "Straight",
            "Slightly curved"
         ],
         "Texture": [
            "Smooth"
         ],
         "Basal_constriction": [
            "Sometimes constricted at basal septum or below it"
         ],
         "Apex": [
            "Obtuse"
         ],
         "Base": [
            "Obconic",
            "With a distinctly protruding hilum"
         ],
         "Septation": [
            "6–20(–26)-distoseptate"
         ],
         "Size": {
            "Length": [
               "80.5–285 µm"
            ],
            "Width": [
               "12.5–27 µm"
            ]
         }
      },
      "Host_plant": [
         "Neoregelia carolinae"
      ]
   }
Send to LLM:

Microcyclospora rumicis ab aliis speciebus generis chlamydosporis numerosis, majoribus,
ad 20 µm diam distinguenda.

Culture characteristics — On MEA slow growing, reaching 3 mm diam after 7 d,
and up to 10 mm after 2 weeks at 25 °C, rais

In [ ]:
# Initial implementation came from a Google search of
# "cosine similarity of two JSON objects python".
# We need to adjust this to handle a 3 level structure of
# (feature, subfeature, value), where the values may be list based, and
# may be categorical. We need to build a dictionary of known features,
# subfeatures, and values to be used to assign numerical values.
import json
import numpy as np
from numpy.linalg import norm

def cosine_similarity_json(json1, json2):
    """
    Calculates the cosine similarity between two JSON objects.

    Args:
        json1 (dict): The first JSON object.
        json2 (dict): The second JSON object.

    Returns:
        float: The cosine similarity between the two JSON objects.
    """
    all_keys = set(json1.keys()) | set(json2.keys())

    vector1 = np.array([json1.get(key, 0) for key in all_keys])
    vector2 = np.array([json2.get(key, 0) for key in all_keys])

    if not np.any(vector1) or not np.any(vector2):
      return 0  # handle zero vector case

    return np.dot(vector1, vector2) / (norm(vector1) * norm(vector2))

# Example Usage
json_string1 = '{"a": 1, "b": 2, "c": 3}'
json_string2 = '{"b": 4, "c": 5, "d": 6}'

json_object1 = json.loads(json_string1)
json_object2 = json.loads(json_string2)

similarity = cosine_similarity_json(json_object1, json_object2)
print(f"Cosine similarity: {similarity}")

In [ ]:
prompt = '''Please extract features, subfeatures, optional subsubfeatures, and values from the following species description.
Format the output as JSON.
The top level of the JSON is feature names. The next level in is subfeature names . The optional next level in is subsubfeature names.
The innermost layer is lists of string-valued values.
Lists are only present at the innermost level of the JSON.
Feature values that are comma-separated strings should be broken down into separate values.
Translate Latin paragraphs to English.
'''

### The JSON keys are by feature, further broken down by subfeature (make sure to distinguish Type from Shape) and further broken down by optional subsubfeature, with lists of string values at the innermost layer.

description = """Fungus anamorphicus. Coloniae in substrato naturali eﬀusae, nigrae. Mycelium
superﬁciale, ex hyphis ramosis, septatis, pallide brunneis vel brunneis, laevibus, 1.5–3
μm crassis compositum. Conidiophora nulla vel brevis, 1–3-septata, brunnea vel
atrobrunnea, 11–28 × 4.5–5 μm. Cellula conidiogena monoblastica, determinatae,
solitaria, simplicia, lageniformia vel ampulliformia, brunnea vel atrobrunnea, laevia,
4.5–6.5 × 3.5–5 μm, ad apicem 3–4.5 μm crassa et truncatae. Conidiorum secessio
schizolytica. Conidia holoblastica, solitaria, acrogena, recta vel curvata, obclavata vel
obclavata-rostrata, atrobrunnea vel brunnea, laevia, 13–19-distoseptata, 130–190 μm
longa, 7–9 μm crassa, apicem versus ad 2–3 μm attenuata; cellula apicalis rotundata;
cellula basalis cylindrica vel conico-truncata, ad basim 3.5–4.5 μm crassa; Appendicibus
lateralibus 0–2, brunneae, septata, cylindricae, surgentibus ex cellulla e apicem 2nd vel
3rd.

Anamorphic fungi. Colonies on natural substrate eﬀuse, black. Mycelium
superﬁcial, composed of branched, septate, pale brown to brown, smoothwalled hyphae, 1.5–3 μm thick. Conidiophores absent or short, 1–3-septate,
brown to dark brown, 11–28 × 4.5–5 μm. Conidiogenous cells monoblastic,
determinate, solitary, simple, lageniform or ampulliform, brown to dark brown,
smooth, 4.5–6.5 × 3.5–5 μm, 3–4.5 μm wide at the truncate apex. Conidial
secession schizolytic. Conidia holoblastic, solitary, acrogenous, straight or
curved, obclavate to obclavate-rostrate, dark brown to brown, smooth, 13–19distoseptate, 130–190 μm long, 7–9 μm thick in the broadest part, tapering
to 2–3 μm near the apex; apical cells rounded; basal cell cylindrical, truncate,
3.5–4.5 μm wide; lateral appendages 0–2, brown, septate, cylindrical, arising
from the 2nd or 3rd cells from the apex.
"""

response = ollama.chat(model=preferred_model, messages=[{
    'role': 'user',
    'content': f'{prompt}:\n\n{description}'},
])
print('Result:')
print(response['message']['content'])